In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 파일 경로 및 데이터 로드
file_path = '/content/CA4Udata.xlsx'
df = pd.read_excel(file_path)

# 필요한 칼럼만 선택 (수정된 칼럼으로 변경)
columns_to_use = ['Name', 'Category', 'Summary-1', 'Summary-2', 'Introduction',
                  'Fee', 'Date', 'Time', 'Place', 'Qualification', 'Method', 'Period',
                  'Affilation-1', 'Affilation-2', 'FeePeriod', 'Membership']
data = df[columns_to_use].copy()

# 불용어 리스트 (필요 시 수정)
stopwords = ['중앙대', '중앙대학교', '동아리', 'PM', '명', '기수', '기존', '기존부원', '신입', '신입부원', '1학기', '00', '정기모임', '기준', '약', '2024', '부원', '여']

# 전처리 함수
def preprocess_text(text):
    text = str(text)  # 문자열로 변환
    for stopword in stopwords:
        text = text.replace(stopword, '')
    return text

# 각 칼럼에 전처리 적용
for col in data.columns[1:]:
    data[col] = data[col].apply(preprocess_text)

# TF-IDF 벡터화 및 토큰 추출
tfidf_matrix = {}
tokens = {}

for col in data.columns[1:]:  # '단체명'은 제외하고 벡터화
    vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")  # 한 글자 단위도 포함
    tfidf_matrix[col] = vectorizer.fit_transform(data[col])
    tokens[col] = vectorizer.get_feature_names_out()

# 코사인 유사도 계산
cosine_sim = {}

for col in data.columns[1:]:
    cosine_sim[col] = cosine_similarity(tfidf_matrix[col])

# 가중치 적용 (수정된 가중치)
weights = {
    'Category': 0.5,
    'Summary-1': 0.5,
    'Summary-2': 0.5,
    'Introduction': 0.5,
    'Fee': 0.3,
    'Date': 0.3,
    'Time': 0.3,
    'Place': 0.3,
    'Qualification': 0.3,
    'Method': 0.3,
    'Period': 0.3,
    'Affilation-1': 0.1,
    'Affilation-2': 0.1,
    'FeePeriod': 0.1,
    'Membership': 0.1
}

# 최종 유사도 계산
final_similarity = np.zeros_like(cosine_sim[data.columns[1]])

for col in data.columns[1:]:
    final_similarity += weights[col] * cosine_sim[col]

# 추천 시스템 구현
def recommend_clubs(selected_club, top_n=3):
    idx = data[data['Name'] == selected_club].index[0]
    sim_scores = list(enumerate(final_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = [x for x in sim_scores if x[0] != idx]  # 선택한 단체 제외
    top_clubs = [data['Name'][i[0]] for i in sim_scores[:top_n]]
    return top_clubs

# 사용 예시
selected_club = '피카통'
recommended_clubs = recommend_clubs(selected_club)
print(f"'{selected_club}'와 유사한 추천 동아리: {recommended_clubs}")